In [ ]:
from __future__ import annotations

import sys

# Add the local directory to the Python path
sys.path.append('/workspaces/tubular/')

import datetime
import warnings
import zoneinfo
from typing import TYPE_CHECKING

import narwhals as nw
import narwhals.selectors as ncs
import numpy as np
import pandas as pd

from tubular.base import BaseTransformer
from tubular.mixins import DropOriginalMixin, NewColumnNameMixin, TwoColumnMixin

if TYPE_CHECKING:
    from narhwals.typing import FrameT

TIME_UNITS = ["us", "ns", "ms"]
TIME_ZONES = zoneinfo.available_timezones().union({None})

DATETIME_VARIANTS = [
    nw.Datetime(time_unit=time_unit, time_zone=time_zone)
    for time_unit in TIME_UNITS
    for time_zone in TIME_ZONES
]

In [ ]:
import pandas as pd
import numpy as np
import polars as pl
import tests.utils as u

In [ ]:
def create_date_diff_different_dtypes_and_nans(library="pandas"):
    """Dataframe with different datetime formats with nans in the data"""

    df_dict = {
        "date_col_1": [
            None,
            datetime.date(2000, 3, 19),
            datetime.date(2018, 11, 10),
            datetime.date(2018, 10, 10),
            datetime.date(2018, 10, 10),
            datetime.date(2018, 10, 10),
            datetime.date(2018, 12, 10),
            datetime.date(
                1985,
                7,
                23,
            ),
        ],
        "date_col_2": [
            datetime.date(2020, 5, 1),
            datetime.date(2019, 12, 25),
            datetime.date(2018, 11, 10),
            datetime.date(2018, 11, 10),
            datetime.date(2018, 9, 10),
            datetime.date(2015, 11, 10),
            datetime.date(2015, 11, 10),
            datetime.date(2015, 7, 23),
        ],
        "datetime_col_1": [
            datetime.datetime(1993, 9, 27, tzinfo=datetime.timezone.utc),
            datetime.datetime(2000, 3, 19, tzinfo=datetime.timezone.utc),
            datetime.datetime(2018, 11, 10, tzinfo=datetime.timezone.utc),
            datetime.datetime(2018, 10, 10, tzinfo=datetime.timezone.utc),
            datetime.datetime(2018, 10, 10, tzinfo=datetime.timezone.utc),
            datetime.datetime(2018, 10, 10, tzinfo=datetime.timezone.utc),
            datetime.datetime(2018, 12, 10, tzinfo=datetime.timezone.utc),
            datetime.datetime(
                1985,
                7,
                23,
                tzinfo=datetime.timezone.utc,
            ),
        ],
        "datetime_col_2": [
            None,
            datetime.datetime(2019, 12, 25, tzinfo=datetime.timezone.utc),
            datetime.datetime(2018, 11, 10, tzinfo=datetime.timezone.utc),
            datetime.datetime(2018, 11, 10, tzinfo=datetime.timezone.utc),
            datetime.datetime(2018, 9, 10, tzinfo=datetime.timezone.utc),
            datetime.datetime(2015, 11, 10, tzinfo=datetime.timezone.utc),
            datetime.datetime(2015, 11, 10, tzinfo=datetime.timezone.utc),
            datetime.datetime(2015, 7, 23, tzinfo=datetime.timezone.utc),
        ],
    }

    return u.dataframe_init_dispatch(df_dict, library=library)

def expected_date_diff_df_2(library="pandas"):
    """Expected output for test_expected_output_drop_cols_true."""

    df_dict = {
        "c": [
            None,
            19,
            0,
            0,
            0,
            -2,
            -3,
            30,
        ],
    }

    return u.dataframe_init_dispatch(df_dict, library=library)

In [ ]:
df_pol = create_date_diff_different_dtypes_and_nans(library="polars")
df_pd = create_date_diff_different_dtypes_and_nans(library="pandas")
df_exp_pol = expected_date_diff_df_2(library="polars")
df_exp_pd = expected_date_diff_df_2(library="pandas")

In [ ]:
def fun(
        X,
        columns : list,
        missing_replacement : str,
        new_column_name : str):
    X = nw.from_native(X)
    
    X = X.with_columns(
        (nw.col(columns[0]).cast(nw.Date).dt.year().cast(nw.Int64) * 10000 +
        nw.col(columns[0]).cast(nw.Date).dt.month().cast(nw.Int64) * 100 +
        nw.col(columns[0]).cast(nw.Date).dt.day().cast(nw.Int64)).alias("col0")
    )
    X = X.with_columns(
        (nw.col(columns[1]).cast(nw.Date).dt.year().cast(nw.Int64) * 10000 +
        nw.col(columns[1]).cast(nw.Date).dt.month().cast(nw.Int64) * 100 +
        nw.col(columns[1]).cast(nw.Date).dt.day().cast(nw.Int64)).alias("col1")
    )

    X = X.with_columns(
        nw.when(nw.col("col1") < nw.col('col0'))
        .then(((nw.col('col0')-nw.col('col1'))//10000)*(-1))
        .otherwise((nw.col('col1')-nw.col('col0'))//10000)
        .cast(nw.Int64)
        .alias(new_column_name)
    )
    
    if missing_replacement is not None:
        X = X.with_columns(
            nw.when(
                (nw.col(columns[0]).is_null())
                or (nw.col(columns[1]).is_null())
            ).then(
                missing_replacement
            ).otherwise(
                nw.col(new_column_name)
            ).cast(nw.Int64).alias(new_column_name)
        )

    return (X.drop(["col0","col1"]))

Traceback (most recent call last):
  File "/home/vscode/.vscode-remote/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 13
    X = X.with_columns(
    ^
SyntaxError: invalid syntax



In [ ]:
fun(X=df_pd,columns=["date_col_1","date_col_1"],missing_replacement=None,new_column_name='NEW').to_native()

   date_col_1 date_col_2  ...            datetime_col_2   NEW
0        <NA> 2020-05-01  ...                       NaT  <NA>
1  2000-03-19 2019-12-25  ... 2019-12-25 00:00:00+00:00     0
2  2018-11-10 2018-11-10  ... 2018-11-10 00:00:00+00:00     0
3  2018-10-10 2018-11-10  ... 2018-11-10 00:00:00+00:00     0
4  2018-10-10 2018-09-10  ... 2018-09-10 00:00:00+00:00     0
5  2018-10-10 2015-11-10  ... 2015-11-10 00:00:00+00:00     0
6  2018-12-10 2015-11-10  ... 2015-11-10 00:00:00+00:00     0
7  1985-07-23 2015-07-23  ... 2015-07-23 00:00:00+00:00     0

[8 rows x 5 columns]


In [ ]:
fun(X=df_pd,columns=["date_col_1","date_col_1"],missing_replacement=None,new_column_name='NEW').to_native()

   date_col_1 date_col_2  ...            datetime_col_2   NEW
0        <NA> 2020-05-01  ...                       NaT  <NA>
1  2000-03-19 2019-12-25  ... 2019-12-25 00:00:00+00:00     0
2  2018-11-10 2018-11-10  ... 2018-11-10 00:00:00+00:00     0
3  2018-10-10 2018-11-10  ... 2018-11-10 00:00:00+00:00     0
4  2018-10-10 2018-09-10  ... 2018-09-10 00:00:00+00:00     0
5  2018-10-10 2015-11-10  ... 2015-11-10 00:00:00+00:00     0
6  2018-12-10 2015-11-10  ... 2015-11-10 00:00:00+00:00     0
7  1985-07-23 2015-07-23  ... 2015-07-23 00:00:00+00:00     0

[8 rows x 5 columns]


In [ ]:
type(df_pd)==pd.core.frame.DataFrame

True
